In [1]:
import pandas as pd
import json
import requests
from time import sleep

# Get Data from API

In [2]:
data = []

#query = "q=&deepsearch=on&declaration_year=2019&doc_type=Після+звільнення"
query = "q=податкова&declaration_year=2019&doc_type=Щорічна"


print("Fetching page #%s" % 1)

r = requests.get("http://declarations.com.ua/search?"+ query + "&format=opendata").json()
data += r["results"]["object_list"]

Fetching page #1


In [3]:
for page in range(2, r["results"]["paginator"]["num_pages"] + 1):
    sleep(0.5)
    print("Fetching page #%s" % page)

    subr = requests.get(
        "http://declarations.com.ua/search?"+ query + "&format=opendata&page=%s" % page).json()
    data += subr["results"]["object_list"]

print("Declarations exported %s" % len(data))

with open("feed.json", "w") as fp:
    json.dump(data, fp)

Fetching page #2
Fetching page #3
Fetching page #4
Fetching page #5
Fetching page #6
Fetching page #7
Fetching page #8
Fetching page #9
Fetching page #10
Fetching page #11
Fetching page #12
Fetching page #13
Fetching page #14
Fetching page #15
Fetching page #16
Fetching page #17
Fetching page #18
Fetching page #19
Fetching page #20
Fetching page #21
Fetching page #22
Fetching page #23
Fetching page #24
Fetching page #25
Fetching page #26
Fetching page #27
Fetching page #28
Fetching page #29
Fetching page #30
Fetching page #31
Fetching page #32
Fetching page #33
Fetching page #34
Fetching page #35
Fetching page #36
Fetching page #37
Fetching page #38
Fetching page #39
Fetching page #40
Fetching page #41
Fetching page #42
Fetching page #43
Fetching page #44
Fetching page #45
Fetching page #46
Fetching page #47
Fetching page #48
Fetching page #49
Fetching page #50
Fetching page #51
Fetching page #52
Fetching page #53
Fetching page #54
Fetching page #55
Fetching page #56
Fetching page #57


In [4]:
print(len(data))

7444


In [5]:
declaration = data[0] #json.loads()
declaration.keys()

dict_keys(['guid', 'infocard', 'raw_source', 'unified_source', 'related_entities'])

In [7]:
declarations = []
not_2019 = 0
corrected = 0
not_NACP = 0
not_yearly = 0

# FILTER OUT ALL DECLARATIONS THAT WERE CORRECTED

for i, declaration in enumerate(data):
    if (not declaration['related_entities']['documents']['corrected']):
        if declaration['infocard']['source'] == 'NACP':
            if (declaration['infocard']['document_type'] == "Щорічна"):
                if (declaration['unified_source']['step_0']['declarationYear1'] == "2019"):
                    declarations.append(declaration)
                else: not_2019 += 1
            else: not_yearly += 1
        else: not_NACP += 1
    else: corrected +=1
        
print(len(data), len(declarations))
print(not_2019, not_yearly, not_NACP, corrected)

7444 6884
0 0 0 560


# Related Entities 

### ОПИС

related_entities — містить інформацію щодо пов'язаних фізичних та юридичних осіб декларанта, а також документів. 
- До фізичних осіб входить родина декларанта.
- До юридичних осіб входять ті компанії, в котрих декларант є бенефіціаром (owned, тільки ЄДРПОУ), має фінансовий зв'язок (related, тільки ЄДРПОУ), а також усі компанії, що були згадані в декларації (all, можуть зустрічатися як ЄДРПОУ так й назви). 
- Нарешті, в пов'язаних документах містяться посилання на наявні виправлені (або оригінальні) декларації до поточної


## Network Based on Owned Companies

In [16]:
if not declarations[0]['related_entities']['companies']['owned']:
    print(1)

1


In [18]:
# Check how many people have 'owned' companies

no_owned = 0
declarations_with_owned_companies = []


for i, declaration in enumerate(declarations):
    if not declaration['related_entities']['companies']['owned']:
        no_owned += 1
    else:
        declarations_with_owned_companies.append(declaration)

        
print(len(declarations))
print(no_owned)
print(len(declarations_with_owned_companies))



6884
6492
392


In [19]:
# Check how mane companies people own and how many people per company we have

companies_full_list = []
people_and_companies = pd.DataFrame({'guid': [], 'relation_type': [], 
                                     'person_name': [], 'office': [], 'position': [],
                                     'document_type': [], 'company_id': []})



for i, declaration in enumerate(declarations_with_owned_companies):
    companies = declaration['related_entities']['companies']['owned']
    result_list = []
    for company in companies:
        companies_full_list.append(company)
        result_list.append(pd.DataFrame([{
            'guid': declaration['infocard']['id'],
            'relation_type': "owned company",
            'person_name': declaration['infocard']['last_name'] + ' ' + declaration['infocard']['first_name'] + ' ' + declaration['infocard']['patronymic'],
            'office': declaration['infocard']['office'], 
            'position': declaration['infocard']['position'],
            'document_type': declaration['infocard']['document_type'], 
            'company_id': company
        }]))
    people_and_companies = people_and_companies.append(result_list)


In [20]:
len(companies_full_list)

530

In [22]:
len(set(companies_full_list))

437

In [23]:
people_and_companies

,guid,relation_type,person_name,office,position,document_type,company_id
0,nacp_7a69033d-cea0-4fae-a4dd-c766ea1f89d1,owned company,Єгорова Тетяна Михайлівна,Державна податкова служба України,начальник відділу методології управління коорд...,Щорічна,00131268
0,nacp_58918925-c53b-4e6f-9ff8-094285f3ed0c,owned company,Ревега Марія Павлівна,Зборівська державна податкова інспекція Кремен...,старший державний інспектор Зборівської держав...,Щорічна,05513862
0,nacp_58918925-c53b-4e6f-9ff8-094285f3ed0c,owned company,Ревега Марія Павлівна,Зборівська державна податкова інспекція Кремен...,старший державний інспектор Зборівської держав...,Щорічна,00278801
0,nacp_9e7a778e-bfaf-4759-9f38-a3d2f02fd39c,owned company,Курус Максим Іванович,Турківська державна податкова інспекція Самбір...,Старший державний інспектор,Щорічна,03482749
0,nacp_350359fc-f103-4a57-912a-fdbbdb99c967,owned company,Румянцева Інна Георгіївна,Головне управління державної податкової служби...,головний державний ревізор-інспектор відділу п...,Щорічна,20262860
...,...,...,...,...,...,...,...
0,nacp_987a304f-ad59-49fb-8ee9-7c7be92e8ada,owned company,Тихонов Сергій Валентинович,Офіс великих платників податків ДПС,Головний державний ревізор-інспектор відділу о...,Щорічна,00017733
0,nacp_19a226fe-be5e-4230-a0fa-d2ad4f4dcd13,owned company,Шальнєва Світлана Миколаївна,Головне управління ДПС у Донецькій області,головний державний ревізор - інспектор відділу...,Щорічна,01558750
0,nacp_024bc560-8e3d-4de5-9c4f-d31195a9488c,owned company,Лучечко Наталія Михайлівна,Головне управління ДПС у Львівській області,Головний державний ревізор-інспектор відділу к...,Щорічна,03349039
0,nacp_fada9071-9754-44e6-8463-33e695cf300d,owned company,Зелінська Аліна Володимирівна,Головне управління ДПС в Івано-Франківській об...,Старший державний інспектор сектору податкових...,Щорічна,00135390


In [24]:
# Most Popular companies -- check how many people are related to the same company

people_and_companies[['company_id','person_name']].groupby(['company_id'])['person_name'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)

,company_id,count
0,00017733,15
31,00213428,7
24,00191230,6
8,00135390,5
194,14358604,4
186,14305909,4
144,05607824,3
211,21541450,3
119,04012721,3
131,05441447,2


In [77]:

people_and_companies[['company_id','person_name']].groupby(['company_id'])['person_name'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(30)

,company_id,count
0,00017733,15
31,00213428,7
24,00191230,6
8,00135390,5
194,14358604,4
186,14305909,4
144,05607824,3
211,21541450,3
119,04012721,3
131,05441447,2


In [47]:
people_and_companies.loc[people_and_companies['company_id'] == '00017733']

,guid,relation_type,person_name,office,position,document_type,company_id
0,nacp_b30bb7c0-5c3a-4d68-84e1-bc681618ace6,owned company,Низенко Лариса Володимирівна,Головне управління Державної податкової служби...,Завідувач сектору ведення облікових справ плат...,Щорічна,00017733
0,nacp_77c01808-8799-4729-8e3f-aab54f0657e7,owned company,Ясніцька Тетяна Сергіївна,Державна податкова служба України,Головний державний інспектор,Щорічна,00017733
0,nacp_52b39bb5-8dab-446b-bd16-23231a7ce5de,owned company,Дорош Сергій Олександрович,Державна податкова служба України,начальник відділу,Щорічна,00017733
0,nacp_2ef2b160-2067-4186-b789-aeb3ee0031aa,owned company,Баранов Руслан Олександрович,Державна податкова служба України,Начальник відділу,Щорічна,00017733
0,nacp_54c63481-db7b-442b-832e-94ebd0e762f3,owned company,Тесленко Юлія Павлівна,Державна податкова служба України,Головний державний інспектор,Щорічна,00017733
0,nacp_2f6aabba-a4e7-49e5-a014-ea614c50958f,owned company,Троценко Наталія Михайлівна,Державна податкова служба України,начальник відділу первинної обробки документів...,Щорічна,00017733
0,nacp_993961c0-80c8-4194-8fae-72203d99244b,owned company,Русскіх Світлана Віталіївна,Державна податкова служба України,головний державний ревізор-інспектор,Щорічна,00017733
0,nacp_195dd02f-f8fa-49fb-8272-3caa2a95272b,owned company,Кравчук Тетяна Анатоліївна,"Державна фіскальна служба України, Державна по...",головний державний інспектор,Щорічна,00017733
0,nacp_2f02f383-d26d-4d03-bff9-2bd40dd3743f,owned company,Мельник Олена Леонідівна,Головне управління Державної податкової служби...,Головний державний ревізор-інспектор,Щорічна,00017733
0,nacp_4e4357c4-d88c-4a99-a94c-7d9df508e892,owned company,Кравчук Тетяна Анатоліївна,"Державна фіскальна служба України, Державна по...",головний державний інспектор,Щорічна,00017733


In [30]:
# check if company codes are correct (for Ukrainian companies must be 8 characters)
people_and_companies['company_id'].str.len().value_counts()

8     525
10      3
9       2
Name: company_id, dtype: int64

In [31]:
pairs_p_c = people_and_companies.drop_duplicates(subset = ['guid','company_id'])
pairs_p_c.head(2)

,guid,relation_type,person_name,office,position,document_type,company_id
0,nacp_7a69033d-cea0-4fae-a4dd-c766ea1f89d1,owned company,Єгорова Тетяна Михайлівна,Державна податкова служба України,начальник відділу методології управління коорд...,Щорічна,00131268
0,nacp_58918925-c53b-4e6f-9ff8-094285f3ed0c,owned company,Ревега Марія Павлівна,Зборівська державна податкова інспекція Кремен...,старший державний інспектор Зборівської держав...,Щорічна,05513862


### Simple Graph of people and owned companies

In [33]:
import networkx as nx
from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.

In [66]:
# Create graph

#subset = pairs_p_c[['guid','company_id']]
subset = pairs_p_c[['person_name','company_id']]

tuples = [tuple(x) for x in subset.values]

G = nx.Graph(tuples)
print(nx.info(G))


Name: 
Type: Graph
Number of nodes: 809
Number of edges: 503
Average degree:   1.2435


In [67]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.0015389981519783622


In [68]:
import matplotlib.pyplot as plt
from pyvis.network import Network
nt = Network(notebook=True)
nt.from_nx(G)
nt.show('nx.html')

## Network Based on Related Companies

In [50]:
# Check how many people have owned or related companies

no_owned_related = 0
declarations_with_owned_related_companies = []


for i, declaration in enumerate(declarations):
    if (not declaration['related_entities']['companies']['owned'] 
        and not declaration['related_entities']['companies']['related']):
        no_owned_related += 1
    else:
        declarations_with_owned_related_companies.append(declaration)

        
print(len(declarations))
print(no_owned_related)
print(len(declarations_with_owned_related_companies))



6884
331
6553


In [55]:
# Check how mane companies people own/are related to and how many people per company we have

companies_owned_full_list = []
companies_related_full_list = []

people_and_companies_owned_related = pd.DataFrame({'guid': [], 'relation_type': [], 
                                     'person_name': [], 'office': [], 'position': [],
                                     'document_type': [], 'company_id': []})



for i, declaration in enumerate(declarations_with_owned_related_companies):
    companies_owned = declaration['related_entities']['companies']['owned']
    companies_related = declaration['related_entities']['companies']['related']

    result_list = []
    
    for company in companies_owned:
        companies_owned_full_list.append(company)
        result_list.append(pd.DataFrame([{
            'guid': declaration['infocard']['id'],
            'relation_type': "owned company",
            'person_name': declaration['infocard']['last_name'] + ' ' + declaration['infocard']['first_name'] + ' ' + declaration['infocard']['patronymic'],
            'office': declaration['infocard']['office'], 
            'position': declaration['infocard']['position'],
            'document_type': declaration['infocard']['document_type'], 
            'company_id': company
        }]))
        
        
    for company in companies_related:
        companies_related_full_list.append(company)
        result_list.append(pd.DataFrame([{
            'guid': declaration['infocard']['id'],
            'relation_type': "related company",
            'person_name': declaration['infocard']['last_name'] + ' ' + declaration['infocard']['first_name'] + ' ' + declaration['infocard']['patronymic'],
            'office': declaration['infocard']['office'], 
            'position': declaration['infocard']['position'],
            'document_type': declaration['infocard']['document_type'], 
            'company_id': company
        }]))
        
    people_and_companies_owned_related = people_and_companies_owned_related.append(result_list)

In [56]:
print(len(set(companies_owned_full_list)))
print(len(companies_owned_full_list))

437
530


In [57]:
print(len(set(companies_related_full_list)))
print(len(companies_related_full_list))

4116
17795


In [58]:
people_and_companies_owned_related.head(5)

,guid,relation_type,person_name,office,position,document_type,company_id
0,nacp_c3c6942a-8643-4b56-ac49-284fce34e79a,related company,Задера Наталя Володимирівна,ГОЛОВНЕ УПРАВЛІННЯ ДЕРЖАВНОЇ ПОДАТКОВОЇ СЛУЖБИ...,СТАРШИЙ ДЕРЖАВНИЙ ПОДАТКОВИЙ ІНСПЕКТОР АРБУЗИН...,Щорічна,39394277
0,nacp_c3c6942a-8643-4b56-ac49-284fce34e79a,related company,Задера Наталя Володимирівна,ГОЛОВНЕ УПРАВЛІННЯ ДЕРЖАВНОЇ ПОДАТКОВОЇ СЛУЖБИ...,СТАРШИЙ ДЕРЖАВНИЙ ПОДАТКОВИЙ ІНСПЕКТОР АРБУЗИН...,Щорічна,43144729
0,nacp_cfc45f5b-61a7-40d4-bc58-05d460b3efe9,related company,Горбуненко Світлана Валеріївна,Таращанська державна податкова інспекція Білоц...,Старший державний інспектор Таращанської держа...,Щорічна,39393260
0,nacp_374fc44f-f39e-46cf-bb6e-2f44cd1f2d44,related company,Скибенко Марина Миколаївна,ВАТУТІНСЬКА ДЕРЖАВНА ПОДАТКОВА ІНСПЕКЦІЯ ЗВЕНИ...,СТАРШИЙ ДЕРЖАВНИЙ ІНСПЕКТОР ВАТУТІНСЬКОЇ ДЕРЖА...,Щорічна,43142920
0,nacp_374fc44f-f39e-46cf-bb6e-2f44cd1f2d44,related company,Скибенко Марина Миколаївна,ВАТУТІНСЬКА ДЕРЖАВНА ПОДАТКОВА ІНСПЕКЦІЯ ЗВЕНИ...,СТАРШИЙ ДЕРЖАВНИЙ ІНСПЕКТОР ВАТУТІНСЬКОЇ ДЕРЖА...,Щорічна,39392109


In [59]:
# Most Popular companies -- check how many people are related to the same company

people_and_companies_owned_related[['company_id','person_name']].groupby(['company_id'])['person_name'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(10)

,company_id,count
4320,43005393,912
3454,39292197,906
3536,39440996,471
3499,39394856,469
4401,43145015,459
4365,43141471,386
1270,21680619,380
3616,39599198,351
4390,43143704,333
3519,39406028,309


In [60]:
pairs_p_c_owned_related = people_and_companies_owned_related.drop_duplicates(subset = ['guid','company_id'])

print(len(people_and_companies_owned_related))
print(len(pairs_p_c_owned_related))

18325
18205


In [61]:
# Create graph

#subset = pairs_p_c[['guid','company_id']]
subset = pairs_p_c_owned_related[['person_name','company_id']]

tuples = [tuple(x) for x in subset.values]

G = nx.Graph(tuples)
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 10714
Number of edges: 17446
Average degree:   3.2567


In [62]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.0003039926735082269


In [69]:
import matplotlib.pyplot as plt
from pyvis.network import Network
nt = Network(notebook=True)
nt.from_nx(G)
nt.show('nx__owned_related.html')

In [71]:
nt.show_buttons(filter_=['physics'])